In [1]:
import json
import requests
import pandas as pd
import folium

In [17]:
# API request : real-time data showing filling rate of Nantes' rent-a-bike stations
link = "https://data.nantesmetropole.fr/api/records/1.0/search/?dataset=244400404_stations-velos-libre-service-nantes-metropole-disponibilites&rows=-1"
r = requests.get(link)
data = json.loads(r.text)

In [18]:
bikes = pd.json_normalize(r.json(),
                  record_path = 'records')

In [19]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   datasetid                     127 non-null    object
 1   recordid                      127 non-null    object
 2   fields.available_bike_stands  127 non-null    int64 
 3   fields.bike_stands            127 non-null    int64 
 4   fields.number                 127 non-null    int64 
 5   fields.address                127 non-null    object
 6   fields.name                   127 non-null    object
 7   fields.bonus                  127 non-null    object
 8   fields.banking                127 non-null    object
 9   fields.contract_name          127 non-null    object
 10  fields.status                 127 non-null    object
 11  fields.available_bikes        127 non-null    int64 
 12  fields.position               127 non-null    object
 13  fields.last_update  

In [20]:
columns_to_keep = ['fields.available_bike_stands', 'fields.bike_stands', 'fields.address', 'fields.name', 'fields.status', 'fields.available_bikes', 'fields.position']
bikes = bikes[columns_to_keep]
bikes.head(2)

,fields.available_bike_stands,fields.bike_stands,fields.address,fields.name,fields.status,fields.available_bikes,fields.position
0,6,14,"18, rue Talensac - 26, rue de Bel Air",071-TALENSAC NORD,OPEN,8,"[47.2218536381884, -1.55894074669074]"
1,18,30,"16, boulevard Jules Verne",120-ROND-POINT DE PARIS,OPEN,12,"[47.234853, -1.534981]"


In [21]:
# Builds a map, adds a marker for each station
# Displays number of availables bikes & bike stands
# Markers' color and icon depend on stations' status and the number of available bikes

centre = [47.217212, -1.543517]     # station "060 - GARE DE NANTES NORD 2"
m = folium.Map(location=centre, zoom_start=14)

for i in range(len(bikes)):
  status = bikes.loc[i]['fields.status']
  velos_dispo = bikes.loc[i]['fields.available_bikes']
  places_dispo = bikes.loc[i]['fields.available_bike_stands']
  loc = bikes.loc[i]['fields.position']
  tip = folium.Tooltip(f"<b>{bikes.loc[i]['fields.name']}</b><br>• {velos_dispo} vélo(s) disponible(s)<br>• {places_dispo} place(s) disponible(s)")
  seuil_max = 3

  if status == 'CLOSED':
    icon = folium.Icon(color='gray', icon='glyphicon-remove')
    tip = folium.Tooltip(f"<b>{bikes.loc[i]['fields.name']}</b><br>• STATION FERMÉE")
  else:
    if velos_dispo == 0:
      icon = folium.Icon(color='red', icon="glyphicon-ban-circle")
    elif velos_dispo <= seuil_max:
      icon = folium.Icon(color="orange", icon="glyphicon-exclamation-sign")
    else:
      icon = folium.Icon(color='darkblue', icon="bicycle", prefix="fa")

  folium.Marker(
    location=loc,
    tooltip = tip,
    icon=icon
    ).add_to(m)

m

In [ ]:
#TODO : how to add a title to a Folium map ? This doesn't seem to work :
#title="TITLE"
#m.get_root().html.add_child(folium.Element(title))